<a href="https://colab.research.google.com/github/sdyahaini/KONEKSI-Research/blob/main/Multi_task_DL_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Step-by-Step Setup Guide: Rice Field Classification with GEE + Google Colab**

**Part 1: Google Earth Engine Setup (Data Acquisition)**

Step 1.1: Authenticate Google Earth Engine

In [ ]:
# Install required packages
!pip install -q earthengine-api geemap

# Import and authenticate
import ee
import geemap

# First time: Authenticate (opens browser for login)
ee.Authenticate()

# Initialize Earth Engine with your project ID
ee.Initialize(project='alih-fungsi-lahan-pertanian') # <-- REPLACE with your Earth Engine project ID

print("✓ Earth Engine initialized successfully!")

✓ Earth Engine initialized successfully!


Step 1.2: Define Study Area

In [ ]:
import geopandas as gpd
import json

# Method 1: Define rectangular area
# Replace with your coordinates [lon_min, lat_min, lon_max, lat_max]
aoi = ee.Geometry.Rectangle([106.0, -7.5, 107.0, -6.5])

# Method 2: Upload shapefile/GeoJSON
# Upload your study area file in Colab Files panel
# Then load it:
"""
from google.colab import files
uploaded = files.upload()  # Upload your .geojson file

# Load the geometry
gdf = gpd.read_file(list(uploaded.keys())<sup>0</sup>)
coords = [list(gdf.geometry<sup>0</sup>.exterior.coords)]
aoi = ee.Geometry.Polygon(coords)
"""

# Visualize on map
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.addLayer(aoi, {'color': 'red'}, 'Study Area')
Map

Map(center=[-7.000085277029473, 106.49999999999983], controls=(WidgetControl(options=['position', 'transparent…

Step 1.3: Extract Sentinel-2 Time Series

In [ ]:
import ee

def get_sentinel2_monthly_composite(aoi, year, month):
    """
    Get monthly Sentinel-2 composite with vegetation indices
    """
    # Define date range
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    # Get Sentinel-2 collection
    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    # Cloud masking function
    def maskS2clouds(image):
        qa = image.select('QA60')
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
            qa.bitwiseAnd(cirrusBitMask).eq(0))
        return image.updateMask(mask)

    # Apply cloud mask and create median composite
    composite = s2.map(maskS2clouds).median().clip(aoi)

    # Select optical bands
    optical = composite.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])

    # Calculate vegetation indices
    ndvi = composite.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = composite.normalizedDifference(['B3', 'B8']).rename('NDWI')
    evi = composite.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': composite.select('B8'),
            'RED': composite.select('B4'),
            'BLUE': composite.select('B2')
        }
    ).rename('EVI')

    # Combine all bands
    final = optical.addBands([ndvi, ndwi, evi])

    return final

# Example: Get data for rice growing season (adjust months for your region)
# Rice typically shows distinct patterns during:
# - Transplanting: High NDWI
# - Growing: Increasing NDVI
# - Maturity: High NDVI, decreasing NDWI

year = 2024
months = [3, 4, 5, 6, 7, 8]  # Adjust for your region's rice calendar

composites = []
for month in months:
    composite = get_sentinel2_monthly_composite(aoi, year, month)
    composites.append(composite)
    print(f"✓ Created composite for {year}-{month:02d}")

✓ Created composite for 2024-03
✓ Created composite for 2024-04
✓ Created composite for 2024-05
✓ Created composite for 2024-06
✓ Created composite for 2024-07
✓ Created composite for 2024-08


Step 1.4: Export Data to Google Drive

In [ ]:
def export_to_drive(image, description, folder='rice_field_data', scale=10):
    """
    Export image to Google Drive
    """
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=aoi,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        crs='EPSG:4326'
    )
    task.start()
    print(f"Export started: {description}")
    print(f"Task ID: {task.id}")
    return task

# Export all monthly composites
tasks = []
for i, (month, composite) in enumerate(zip(months, composites)):
    description = f'sentinel2_{year}_{month:02d}'
    task = export_to_drive(composite, description)
    tasks.append(task)

print(f"\n✓ Started {len(tasks)} export tasks")
print("Check progress at: https://code.earthengine.google.com/tasks")

Export started: sentinel2_2024_03
Task ID: XD3YPSFOLQPQHZAOFMICBW4N
Export started: sentinel2_2024_04
Task ID: GJ2QTYO5WFWJH4HV5IBLZQQZ
Export started: sentinel2_2024_05
Task ID: 5BGKCZCMV2J4L2OUXVBUX4MO
Export started: sentinel2_2024_06
Task ID: HX6F7773CPRHTZ4GK4QDTKMG
Export started: sentinel2_2024_07
Task ID: YI2Z6H24UUXJ7M7VMHBPPRLU
Export started: sentinel2_2024_08
Task ID: 4FV5STQOKEFO6OB45HOS5IZV

✓ Started 6 export tasks
Check progress at: https://code.earthengine.google.com/tasks


Step 1.5: Monitor Export Progress

In [ ]:
import time

def check_task_status(tasks):
    """
    Check status of export tasks
    """
    for task in tasks:
        status = task.status()
        state = status['state']
        description = status.get('description', 'Unknown')
        print(f"{description}: {state}")

        if state == 'FAILED':
            print(f"  Error: {status.get('error_message', 'Unknown error')}")

# Check status every minute
while True:
    print("\n" + "="*60)
    check_task_status(tasks)

    # Check if all tasks are done
    states = [task.status()['state'] for task in tasks]
    if all(state in ['COMPLETED', 'FAILED'] for state in states):
        print("\n✓ All tasks finished!")
        break

    print("\nWaiting 60 seconds...")
    time.sleep(60)

**Part 2: Google Colab Setup (Deep Learning)**

Step 2.1: Mount Google Drive and Setup Environment

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install deep learning packages
!pip install -q segmentation-models-pytorch albumentations opencv-python-headless
!pip install -q rasterio geopandas scikit-image tqdm

# Check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

MessageError: Error: credential propagation was unsuccessful

Step 2.2: Organize Your Data

In [ ]:
import os
from pathlib import Path
import shutil

# Create project structure
project_dir = Path('/content/rice_field_project')
project_dir.mkdir(exist_ok=True)

# Create subdirectories
(project_dir / 'data' / 'images').mkdir(parents=True, exist_ok=True)
(project_dir / 'data' / 'masks').mkdir(parents=True, exist_ok=True)
(project_dir / 'models').mkdir(exist_ok=True)
(project_dir / 'results').mkdir(exist_ok=True)

# Copy data from Google Drive
drive_data_dir = Path('/content/drive/MyDrive/rice_field_data')
local_data_dir = project_dir / 'data' / 'images'

# Copy exported GeoTIFFs
for tif_file in drive_data_dir.glob('*.tif'):
    shutil.copy(tif_file, local_data_dir)
    print(f"Copied: {tif_file.name}")

print(f"\n✓ Data organized in: {project_dir}")

Step 2.3: Prepare Training Labels

Step 2.4:

Step 2.5:

Step 2.6:

Step 2.7:

Step 2.8:

Step 2.9:

**Part 3: Inference and Evaluation**

Step 3.1: Load Best Model

Step 3.2: Run Inference

Step 3.3: Save and Visualize Results

Step 3.4: Calculate Metrics (if ground truth available)

**Part 4: Deployment and Scaling**

Step 4.1: Save Model for Future Use

Step 4.2 Batch Processing Multiple Images

**Part 5: Tips and Bes Practices**